In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from cmath import sqrt

# Data Visualization

In [2]:
train_data = pd.read_csv("https://raw.githubusercontent.com/cdavidshaffer/CPSC4970-AI/master/data/m3train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/cdavidshaffer/CPSC4970-AI/master/data/m3test.csv")

In [3]:
display(train_data)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,0.003555,92.159562,6.658918,0.0,1.994330,35.960534,290.003375,20.973961,7.523349,2410.680571,33.715837,1965.274105,34.370406,220.134692
1,0.015362,0.000000,20.380324,0.0,1.738552,35.118264,350.939667,25.471824,15.046698,1970.894250,39.224961,1965.274105,63.081429,198.121223
2,0.015351,0.000000,20.380324,0.0,1.738552,39.296796,271.766966,25.471824,15.046698,1970.894250,39.224961,1945.121257,27.813803,318.278075
3,0.018208,0.000000,6.284173,0.0,1.697776,38.274040,203.714027,31.087615,22.570047,1808.010428,41.208245,1954.034064,20.290963,306.354113
4,0.038841,0.000000,6.284173,0.0,1.697776,39.088964,241.076425,31.087615,22.570047,1808.010428,41.208245,1965.274105,36.785997,332.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0.019896,174.079173,17.555329,0.0,1.605102,36.042573,179.695343,28.162030,52.663442,2679.438878,35.478757,1959.579811,65.566036,201.790134
302,0.052122,174.079173,17.555329,0.0,1.605102,35.522991,81.841443,28.162030,52.663442,2679.438878,35.478757,1899.467875,59.837635,242.148161
303,0.056251,174.079173,17.555329,0.0,1.605102,38.186532,78.727910,28.162030,52.663442,2679.438878,35.478757,1933.237513,33.542204,303.602429
304,0.031022,168.959198,6.284173,0.0,1.749673,39.575730,182.808876,20.625249,52.663442,1808.010428,40.547150,1949.330082,47.828698,331.119266


In [4]:
train_data.info()
train_data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       306 non-null    float64
 1   B       306 non-null    float64
 2   C       306 non-null    float64
 3   D       306 non-null    float64
 4   E       306 non-null    float64
 5   F       306 non-null    float64
 6   G       306 non-null    float64
 7   H       306 non-null    float64
 8   I       306 non-null    float64
 9   J       306 non-null    float64
 10  K       306 non-null    float64
 11  L       306 non-null    float64
 12  M       306 non-null    float64
 13  N       306 non-null    float64
dtypes: float64(14)
memory usage: 33.6 KB


,A,B,C,D,E,F,G,H,I,J,K,L,M,N
count,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000
mean,0.240841,81.216869,25.312196,0.610402,1.901232,35.155155,277.079741,22.044135,34.076345,2539.124578,38.717977,1876.419011,73.442306,234.288777
std,0.374535,133.126869,18.321462,1.965383,0.401590,3.839247,129.633241,10.131636,12.292388,560.974481,5.012664,212.617866,42.569868,81.040005
min,0.003555,0.000000,1.326018,0.000000,1.427170,26.815893,12.898923,6.777307,7.523349,1531.107930,27.765983,350.570438,11.939920,108.232890
25%,0.037393,0.000000,11.674726,0.000000,1.619930,32.508049,162.793305,13.610511,30.093396,2150.066455,34.542206,1892.535691,41.030535,179.776665
50%,0.076583,0.000000,18.477776,0.000000,1.812691,34.325218,294.228884,20.313204,33.855070,2500.266673,39.224961,1942.076047,65.566036,212.796869
75%,0.291414,112.639465,30.527246,0.000000,2.019352,37.166511,405.204102,29.337392,37.616744,3127.369389,42.089705,1958.936109,97.054988,272.875295
max,2.304820,511.997569,73.939931,6.917893,3.228739,47.719492,444.790452,47.296025,60.186791,3819.625634,46.717369,1965.274105,237.487085,458.613941


In [5]:
display(test_data)

,A,B,C,D,E,F,G,H,I,J,K,L,M
0,0.042205,168.959198,6.284173,0.0,1.749673,40.582078,319.804335,15.893032,52.663442,1808.010428,40.54715,1965.274105,44.653922
1,0.027743,168.959198,6.284173,0.0,1.749673,37.459117,312.687688,16.321229,52.663442,1808.010428,40.54715,1965.274105,51.969711
2,0.277305,0.000000,28.538219,0.0,2.016572,36.288691,366.952123,17.012498,30.093396,2475.834100,40.54715,1965.274105,31.333664
3,0.196540,0.000000,28.538219,0.0,2.016572,32.662556,341.154277,15.909954,30.093396,2475.834100,40.54715,1962.006076,68.809830
4,1.482478,0.000000,28.538219,0.0,2.016572,27.198743,168.130791,12.919755,30.093396,2475.834100,40.54715,1735.274150,87.237337
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8.007976,0.000000,52.175936,0.0,2.568905,34.691660,444.790452,8.072154,180.560373,5424.031284,44.51372,1965.274105,140.242301
96,5.397663,0.000000,52.175936,0.0,2.568905,35.025287,444.790452,8.404969,180.560373,5424.031284,44.51372,1862.331176,140.173284
97,13.951154,0.000000,52.175936,0.0,2.568905,29.255193,426.998834,8.732142,180.560373,5424.031284,44.51372,1965.274105,136.446372
98,23.360505,0.000000,52.175936,0.0,2.568905,30.250603,379.851046,8.242920,180.560373,5424.031284,44.51372,1631.340909,188.968218


In [6]:
test_data.info()
test_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       100 non-null    float64
 1   B       100 non-null    float64
 2   C       100 non-null    float64
 3   D       100 non-null    float64
 4   E       100 non-null    float64
 5   F       100 non-null    float64
 6   G       100 non-null    float64
 7   H       100 non-null    float64
 8   I       100 non-null    float64
 9   J       100 non-null    float64
 10  K       100 non-null    float64
 11  L       100 non-null    float64
 12  M       100 non-null    float64
dtypes: float64(13)
memory usage: 10.3 KB


,A,B,C,D,E,F,G,H,I,J,K,L,M
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,4.232633,45.874982,34.251340,0.553431,2.196833,32.974305,318.114131,19.179974,105.703052,3950.421343,42.896241,1894.602967,95.350271
std,7.490217,113.673570,19.039490,1.886232,0.420852,4.064642,123.556988,13.807749,75.585087,1522.374716,2.683456,157.974747,57.157989
min,0.007318,0.000000,3.603310,0.000000,1.519843,19.476116,65.384196,5.792711,7.523349,1522.963739,34.156567,650.733996,20.428997
25%,0.037673,0.000000,14.960945,0.000000,1.827518,31.239176,204.937201,8.136769,30.093396,2475.834100,40.547150,1872.766588,50.416831
50%,1.718394,0.000000,40.357077,0.000000,2.177824,33.204015,368.508890,13.229749,116.611908,4463.016732,44.513720,1950.320394,79.300395
75%,5.244347,0.000000,52.175936,0.000000,2.568905,35.100489,431.446739,27.062563,180.560373,5424.031284,44.513720,1965.274105,134.375866
max,50.049821,460.797812,52.175936,6.917893,2.854339,48.020302,444.790452,62.185999,180.560373,5424.031284,48.480289,1965.274105,262.057094


# Data Splitting

In [7]:
x_train = train_data.drop(columns=['N'])
y_train = train_data['N']

# Create Cross-Validation Linear Model

In [20]:
pl = Pipeline([('norm', StandardScaler()),
               ('poly', PolynomialFeatures()),
               ('regr', TransformedTargetRegressor(transformer=StandardScaler()))])

param_grid = [
    {'poly__degree': [1, 2, 3, 4, 5, 6], 'regr__regressor': [Lasso(max_iter=100000)], 'regr__regressor__alpha': [0.01, 0.05, 0.1, 0.5, 1]},
    {'poly__degree': [1, 2, 3, 4, 5, 6], 'regr__regressor': [Ridge()], 'regr__regressor__alpha': [0.01, 0.05, 0.1, 0.5, 1]},
    {'poly__degree': [1, 2, 3, 4, 5, 6], 'regr__regressor': [LinearRegression()]},
]

model = GridSearchCV(pl, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1)
model.fit(x_train, y_train)
pred_train = model.predict(x_train)
pred_test= model.predict(test_data)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.969e-01, tolerance: 2.440e-02
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.632e-02, tolerance: 2.450e-02
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.925e-01, toler

Exception ignored in: <function ResourceTracker.__del__ at 0x1031a9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


In [21]:
print("Training RMSE: ", sqrt(mean_squared_error(y_train, pred_train)))

Training RMSE:  (31.409046356034196+0j)


All predictions are based on 

In [22]:
print("Best parameters found: ", model.best_params_)
# print("Best cross-validation score (MSE): ", -model.best_score_)

Best parameters found:  {'poly__degree': 1, 'regr__regressor': Lasso(max_iter=100000), 'regr__regressor__alpha': 0.05}


In [ ]:
#best_model = model.best_estimator_
# SGDRegressor(max_iter=1000, tol=1e-3)